# Item-based Collaborative Filtering

Цель:
- построить item-based рекомендательную модель на основе совместных покупок (co-occurrence);
- сгенерировать персональные рекомендации для пользователей;
- оценить качество по `precision@k` и `recall@k` на тестовой выборке;
- сравнить результаты с popularity-baseline.


In [14]:
import pandas as pd
import numpy as np
from collections import defaultdict

usecols = ["visitorid", "itemid", "event", "timestamp"]

train = pd.read_csv("../data/processed/train.csv", usecols=usecols)
test = pd.read_csv("../data/processed/test.csv", usecols=usecols)

for df in (train, test):
    df["timestamp"] = pd.to_datetime(df["timestamp"])

train.shape, test.shape, train["event"].value_counts()


((1008982, 4),
 (126123, 4),
 event
 view           945976
 addtocart       46036
 transaction     16970
 Name: count, dtype: int64)

In [15]:
train_trans = train[train["event"] == "transaction"]
test_trans = test[test["event"] == "transaction"]

len(train_trans), len(test_trans)


(16970, 2254)

In [16]:
user2items_train = (
    train_trans
    .groupby("visitorid")["itemid"]
    .apply(list)
)

user2items_train.head()


visitorid
172    [465522, 10034]
186            [49029]
419            [19278]
539            [94371]
795           [207825]
Name: itemid, dtype: object

In [17]:
item_cooc: dict[int, dict[int, int]] = defaultdict(lambda: defaultdict(int))

for items in user2items_train:
    # берём уникальные товары, чтобы не дублировать пары
    unique_items = set(items)
    for i in unique_items:
        for j in unique_items:
            if i != j:
                item_cooc[i][j] += 1


In [18]:
sample_item = next(iter(item_cooc))
sorted(item_cooc[sample_item].items(), key=lambda x: -x[1])[:5]


[(37029, 3), (213834, 3), (119736, 3), (231482, 2), (268883, 2)]

In [19]:
def recommend_item_based(
    user_items: list[int],
    item_cooc: dict[int, dict[int, int]],
    top_k: int = 20,
) -> list[int]:
    """
    user_items: товары, с которыми взаимодействовал пользователь (train)
    item_cooc: матрица co-occurrence по товарам
    """
    scores: dict[int, int] = defaultdict(int)

    for item in user_items:
        for related_item, cnt in item_cooc.get(item, {}).items():
            scores[related_item] += cnt

    # не рекомендуем уже купленные товары
    for item in user_items:
        scores.pop(item, None)

    ranked_items = sorted(scores.items(), key=lambda x: -x[1])
    return [item for item, _ in ranked_items[:top_k]]


In [20]:
test_user2items = (
    test_trans
    .groupby("visitorid")["itemid"]
    .apply(set)
)

len(test_user2items)


1198

In [21]:
from typing import Iterable, Set, Tuple

def precision_recall_at_k(
    recommended: Iterable[int],
    relevant: Set[int],
    k: int,
) -> Tuple[float, float]:
    if k == 0:
        return 0.0, 0.0

    rec_k = list(recommended)[:k]
    rec_set = set(rec_k)

    hit_count = len(rec_set & relevant)
    precision = hit_count / k
    recall = hit_count / len(relevant) if relevant else 0.0

    return precision, recall


In [22]:
def evaluate_item_based(
    user2items_test: pd.Series,
    user2items_train: pd.Series,
    item_cooc: dict[int, dict[int, int]],
    ks=(5, 10, 20),
) -> pd.DataFrame:
    rows = []

    for k in ks:
        precisions = []
        recalls = []

        for user, relevant_items in user2items_test.items():
            # берём только тех, у кого есть история в train
            if user not in user2items_train:
                continue

            user_items = user2items_train[user]
            recs = recommend_item_based(user_items, item_cooc, top_k=k)

            p, r = precision_recall_at_k(recs, relevant_items, k)
            precisions.append(p)
            recalls.append(r)

        rows.append({
            "k": k,
            "precision@k": np.mean(precisions) if precisions else 0.0,
            "recall@k": np.mean(recalls) if recalls else 0.0,
        })

    return pd.DataFrame(rows)


In [23]:
item_cf_results = evaluate_item_based(
    user2items_test=test_user2items,
    user2items_train=user2items_train,
    item_cooc=item_cooc,
    ks=(5, 10, 20),
)

item_cf_results


,k,precision@k,recall@k
0,5,0.000000,0.000000
1,10,0.002174,0.000418
2,20,0.002174,0.001363


In [24]:
# popularity по train_trans
item_popularity = train_trans["itemid"].value_counts()
TOP_N = 100
top_items = item_popularity.index[:TOP_N].tolist()

def evaluate_popularity_baseline(
    user2items: pd.Series,
    popular_items: list[int],
    ks=(5, 10, 20),
) -> pd.DataFrame:
    rows = []

    for k in ks:
        precisions = []
        recalls = []

        for _, relevant_items in user2items.items():
            p, r = precision_recall_at_k(popular_items, relevant_items, k)
            precisions.append(p)
            recalls.append(r)

        rows.append({
            "k": k,
            "precision@k": np.mean(precisions),
            "recall@k": np.mean(recalls),
        })

    return pd.DataFrame(rows)

pop_results = evaluate_popularity_baseline(
    user2items=test_user2items,
    popular_items=top_items,
    ks=(5, 10, 20),
)

pop_results


,k,precision@k,recall@k
0,5,0.009182,0.021804
1,10,0.005259,0.026569
2,20,0.003297,0.034098


In [25]:
compare = item_cf_results.copy()
compare["precision@k (popularity)"] = pop_results["precision@k"]
compare["recall@k (popularity)"] = pop_results["recall@k"]
compare


,k,precision@k,recall@k,precision@k (popularity),recall@k (popularity)
0,5,0.000000,0.000000,0.009182,0.021804
1,10,0.002174,0.000418,0.005259,0.026569
2,20,0.002174,0.001363,0.003297,0.034098


## Выводы по item-based collaborative filtering

Item-based collaborative filtering был реализован на основе
co-occurrence товаров, купленных одними и теми же пользователями.

### Полученные результаты

- В рамках данного датасета item-based CF показал более низкие значения
  `precision@k` и `recall@k` по сравнению с popularity-based baseline.
- Особенно заметно отставание по recall, что указывает на слабое покрытие
  пользовательских интересов.

### Причины наблюдаемого результата

- Датасет обладает высокой разреженностью:
  большинство пользователей совершают 1–2 покупки.
- Popularity bias в данных чрезвычайно силён и даёт высокий baseline.
- Простая co-occurrence модель не учитывает:
  - относительную популярность товаров,
  - силу пользовательских предпочтений,
  - скрытые факторы взаимодействий.

### Вывод

Item-based CF в данной конфигурации не превосходит popularity baseline,
что является ожидаемым результатом для e-commerce данных с редкими покупками.
Данный эксперимент подчёркивает необходимость использования
более мощных моделей, таких как matrix factorization (implicit ALS).
